In [19]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes/')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp

test_key = 'DB318A59-25FF-499E-9A6D-783A19C346D8'

#free test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)

meta_path='/home/fbuonerba/codes/meta_data/'
with open(meta_path+'top_coins.txt') as f:
    coins=json.load(f)
with open(meta_path+'top_exchanges.txt') as ff:
    exchanges=json.load(ff)

quotes=['USD','BTC']

In [13]:
#coinAPI very bad latest-historical ohlcv#
count=0
for coi in coins:
    for quote in quotes:
        for ex in exchanges:
            sym_id=str(ex)+'_SPOT_'+str(coi)+'_'+str(quote)
            try:
                api.ohlcv_latest_data(sym_id, {'period_id': '1MTH', 'limit': 1})
                print(sym_id, 'success')
            except urllib.error.HTTPError as err:
                print(sym_id, 'error', err.code)
                count+=1

BINANCE_SPOT_BTC_USD error 400
BITFINEX_SPOT_BTC_USD success
HUOBI_SPOT_BTC_USD error 400
COINBASE_SPOT_BTC_USD success
OKEX_SPOT_BTC_USD error 400
BITSTAMP_SPOT_BTC_USD success
HITBTC_SPOT_BTC_USD error 400
BITHUMB_SPOT_BTC_USD error 400
POLONIEX_SPOT_BTC_USD error 400
KRAKEN_SPOT_BTC_USD success
BINANCE_SPOT_BTC_BTC error 400
BITFINEX_SPOT_BTC_BTC error 400
HUOBI_SPOT_BTC_BTC error 400
COINBASE_SPOT_BTC_BTC error 400
OKEX_SPOT_BTC_BTC error 400
BITSTAMP_SPOT_BTC_BTC error 400
HITBTC_SPOT_BTC_BTC error 400
BITHUMB_SPOT_BTC_BTC error 400
POLONIEX_SPOT_BTC_BTC error 400
KRAKEN_SPOT_BTC_BTC error 400
BINANCE_SPOT_ETH_USD error 400
BITFINEX_SPOT_ETH_USD success
HUOBI_SPOT_ETH_USD error 400
COINBASE_SPOT_ETH_USD success
OKEX_SPOT_ETH_USD error 400
BITSTAMP_SPOT_ETH_USD success
HITBTC_SPOT_ETH_USD error 400
BITHUMB_SPOT_ETH_USD error 400
POLONIEX_SPOT_ETH_USD error 400
KRAKEN_SPOT_ETH_USD success
BINANCE_SPOT_ETH_BTC success
BITFINEX_SPOT_ETH_BTC success
HUOBI_SPOT_ETH_BTC error 400
COINBAS

KeyboardInterrupt: 

In [ ]:
#Getting data from CoinMarketCap#
web_data= urllib.request.urlopen('https://api.coinmarketcap.com/v2/ticker/')
cmc_data=web_data.read()
cmc=json.loads(cmc_data.decode('utf-8'))
cmc=cmc['data']
xxx=list(cmc.values())
#diction={}
co=0
for x in xxx:
    if x['symbol'] in coins:
        diction[x['id']]=x['symbol']
        co+=1
print(co, diction)

In [ ]:
#Getting id of coins listed on CoinMarketCap which appear in our top_coins list.
#start=100*n+1, since they limit to 100 coins per request.
web_data1= urllib.request.urlopen('https://api.coinmarketcap.com/v2/ticker/?start=1701')
cmc_data1=web_data1.read()
cmc1=json.loads(cmc_data1.decode('utf-8'))
cmc1=cmc1['data']
co1=0
xxx1=list(cmc1.values())
for x in xxx1:
    if x['symbol'] in coins:
        print(x['symbol'])
        diction[x['id']]=x['symbol']
        co1+=1
print(co1, diction, len(diction))

In [25]:
with open(meta_path+'symbol_list.txt') as ll:
    ll=json.load(ll)

In [26]:
good_symbols=[]
for coin in coins:
    for quote in quotes:
        for exc in exchanges:
            s=str(exc)+'_SPOT_'+str(coin)+'_'+str(quote)
            good_symbols.append(s)
good_symbols

['BINANCE_SPOT_BTC_USD',
 'BITFINEX_SPOT_BTC_USD',
 'HUOBI_SPOT_BTC_USD',
 'COINBASE_SPOT_BTC_USD',
 'OKEX_SPOT_BTC_USD',
 'BITSTAMP_SPOT_BTC_USD',
 'HITBTC_SPOT_BTC_USD',
 'BITHUMB_SPOT_BTC_USD',
 'POLONIEX_SPOT_BTC_USD',
 'KRAKEN_SPOT_BTC_USD',
 'BINANCE_SPOT_BTC_BTC',
 'BITFINEX_SPOT_BTC_BTC',
 'HUOBI_SPOT_BTC_BTC',
 'COINBASE_SPOT_BTC_BTC',
 'OKEX_SPOT_BTC_BTC',
 'BITSTAMP_SPOT_BTC_BTC',
 'HITBTC_SPOT_BTC_BTC',
 'BITHUMB_SPOT_BTC_BTC',
 'POLONIEX_SPOT_BTC_BTC',
 'KRAKEN_SPOT_BTC_BTC',
 'BINANCE_SPOT_ETH_USD',
 'BITFINEX_SPOT_ETH_USD',
 'HUOBI_SPOT_ETH_USD',
 'COINBASE_SPOT_ETH_USD',
 'OKEX_SPOT_ETH_USD',
 'BITSTAMP_SPOT_ETH_USD',
 'HITBTC_SPOT_ETH_USD',
 'BITHUMB_SPOT_ETH_USD',
 'POLONIEX_SPOT_ETH_USD',
 'KRAKEN_SPOT_ETH_USD',
 'BINANCE_SPOT_ETH_BTC',
 'BITFINEX_SPOT_ETH_BTC',
 'HUOBI_SPOT_ETH_BTC',
 'COINBASE_SPOT_ETH_BTC',
 'OKEX_SPOT_ETH_BTC',
 'BITSTAMP_SPOT_ETH_BTC',
 'HITBTC_SPOT_ETH_BTC',
 'BITHUMB_SPOT_ETH_BTC',
 'POLONIEX_SPOT_ETH_BTC',
 'KRAKEN_SPOT_ETH_BTC',
 'BINANCE_SP

In [29]:
final=set(ll)&set(good_symbols)

In [31]:
final=list(final)
final

['BITFINEX_SPOT_DASH_BTC',
 'BITSTAMP_SPOT_LTC_BTC',
 'HITBTC_SPOT_GNT_BTC',
 'BITSTAMP_SPOT_BCH_USD',
 'POLONIEX_SPOT_DOGE_BTC',
 'BITFINEX_SPOT_DASH_USD',
 'HITBTC_SPOT_NEO_BTC',
 'BINANCE_SPOT_ZRX_BTC',
 'COINBASE_SPOT_BCH_USD',
 'BINANCE_SPOT_BNB_BTC',
 'OKEX_SPOT_ETC_BTC',
 'BITFINEX_SPOT_ETC_USD',
 'BITFINEX_SPOT_ODE_BTC',
 'BINANCE_SPOT_QTUM_BTC',
 'BITFINEX_SPOT_BCH_BTC',
 'BITFINEX_SPOT_REP_USD',
 'BITFINEX_SPOT_BTG_BTC',
 'BITFINEX_SPOT_SNT_BTC',
 'KRAKEN_SPOT_DASH_USD',
 'BITSTAMP_SPOT_ETH_BTC',
 'BITFINEX_SPOT_ZEC_BTC',
 'BITSTAMP_SPOT_EUR_USD',
 'BITFINEX_SPOT_LRC_USD',
 'BINANCE_SPOT_TRX_BTC',
 'KRAKEN_SPOT_DASH_BTC',
 'BITFINEX_SPOT_XVG_BTC',
 'OKEX_SPOT_BCH_BTC',
 'KRAKEN_SPOT_USDT_USD',
 'COINBASE_SPOT_LTC_USD',
 'BINANCE_SPOT_LSK_BTC',
 'POLONIEX_SPOT_GNT_BTC',
 'BITFINEX_SPOT_BAT_USD',
 'BITFINEX_SPOT_ZRX_BTC',
 'HITBTC_SPOT_MKR_BTC',
 'BITFINEX_SPOT_DAT_USD',
 'BITSTAMP_SPOT_LTC_USD',
 'BITFINEX_SPOT_IOTA_BTC',
 'BITSTAMP_SPOT_XRP_USD',
 'COINBASE_SPOT_ETH_USD',
 'O

In [32]:
with open(meta_path+'symbols.txt','w') as ffff:
    json.dump(final,ffff)